In [9]:
import pandas as pd 
df = pd.read_excel('库存优化速率计算逻辑.xlsx')
df.head()

,仓库,零件,成本,分类分库速率,单库速率,全网速率,segma速率,segma分类,ABC分类,备货数量,预测数量,分库分类满足率,单库满足率,全网满足率,segma满足率
0,CPD1,Part1,27,0.04,0.04,0.06,0.06,A,A,100,5,0.82,0.87,0.90,0.93
1,CPD1,Part2,26,0.03,0.02,0.04,0.04,A,A,101,5,0.87,0.88,0.84,0.89
2,CPD1,Part3,18,0.10,0.04,0.07,0.07,A,A,102,5,0.86,0.99,0.89,0.96
3,CPD1,Part4,30,0.10,0.09,0.09,0.09,A,B,103,5,0.88,0.80,0.89,0.85
4,CPD1,Part5,19,0.02,0.06,0.02,0.02,A,B,104,5,0.81,0.89,0.96,0.83


In [10]:
# 假设我有一个dataframe ,有如下字段['一级分类', 'sku', '成本', '分类分库速率', '单库速率', 
#                        'sku速率','segma分类','二级分类', '备货数量','分库分类满足率', 
#                        '单库满足率', 'sku满足率','segma满足率']


# 我有多个sku ，每个sku有对应的分类分库类别，单库类别,segma分类，我需要调整sku的备货数量，使得每个分类分库类别，单库类别,segma满足率,sku相应的满足率得到满足，规则如下：
# 1.满足率 = 分组内sku的备货数量*速率系数， 速率有一级速率，segma速率，二级速率，sku速率，系数不同
#     比如单库满足率 = 单库分类下所有sku的满足率求和
      
# 2. 完成调整后，该sku所属的分类分库类别的满足率大于等于表格里分类分库满足率的值
# 3. 完成调整后，该sku所属的单库类别的满足率大于等于表格里单库满足率的值
# 4. 完成调整后，该sku所属的segma类别的满足率大于等于表格里segma满足率的值
# 5. 完成调整后，sku的满足率=所有sku的备货数量*sku速率大于等于表格里全网满足率的值
# 6.每次sku调整会有成本，总成本 = 备货数量*成本
# 7.总成本最小
# 8.备货数量不能为负数


In [11]:
# 假设我有一个dataframe ,有如下字段['一级分类', 'sku', '成本', '二级速率', '一级速率', 
#                        'sku速率', '二级分类', '备货数量','一级分类满足率', '二级分类满足率', 'sku满足率']


# 我有多个sku ，每个sku有对应的一级分类，二级分类，我需要调整sku的备货数量，使得每个一级，二级类别,sku相应的满足率得到满足，规则如下：
# 1.满足率 = 分组内sku的备货数量*速率系数， 速率有一级速率，二级速率，sku速率，系数不同
#     比如一级分类满足率 = 一级分类下所有sku的满足率求和
      

# 2. 完成调整后，该sku所属的二级分类的满足率大于等于表格里二级分类满足率的值
# 3. 完成调整后，该sku所属的一级分类的满足率大于等于表格里一级分类满足率的值
# 4. 完成调整后，sku的满足率=所有sku的备货数量*sku速率
# 5.每次sku调整会有成本，总成本 = 备货数量*成本
# 6.总成本最小
# 7.备货数量不能为负数


In [12]:
import pandas as pd
from pulp import LpProblem, LpVariable, lpSum, LpMinimize, LpStatus

# 创建线性规划问题
prob = LpProblem("Inventory_Optimization", LpMinimize)

# 创建决策变量，每个sku的备货数量为变量，上界设置为None（无上界）
sku_vars = LpVariable.dicts("Inventory", df['零件'], lowBound=0, upBound=None, cat='Integer')

# 定义目标函数（总成本最小化）
prob += lpSum(df['成本'] * sku_vars[sku] for sku in df['零件']), "Total_Cost"

# 定义约束条件
for i, row in df.iterrows():
    # 约束条件 1: 每个sku所属二级分类的满足率大于等于二级分类满足率
    prob += row['分类分库速率'] * sku_vars[row['零件']] >= row['分库分类满足率']

    # 约束条件 2: 每个sku所属一级分类的满足率大于等于一级分类满足率
    prob += row['单库速率'] * sku_vars[row['零件']] >= row['单库满足率']

    # 约束条件 3: sku的满足率等于所有sku的备货数量*sku速率
    prob += row['全网速率'] * sku_vars[row['零件']] >= row['全网满足率']
    
    prob += row['segma速率'] * sku_vars[row['零件']] >= row['segma满足率']


# 解决问题
prob.solve()
df['最优备货数量'] = [round(sku_vars[sku].value()) for sku in df['零件']]

# 输出结果
print("Status:", LpStatus[prob.status])
print("Total Cost:", round(prob.objective.value(), 2))
df

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/kaka/opt/anaconda3/lib/python3.9/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/7a802f96362e48a29e6bb6c72bffda41-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/7a802f96362e48a29e6bb6c72bffda41-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 149 COLUMNS
At line 330 RHS
At line 475 BOUNDS
At line 488 ENDATA
Problem MODEL has 144 rows, 12 columns and 144 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 2.2572e+06 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 2.2572e+06 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of

,仓库,零件,成本,分类分库速率,单库速率,全网速率,segma速率,segma分类,ABC分类,备货数量,预测数量,分库分类满足率,单库满足率,全网满足率,segma满足率,最优备货数量
0,CPD1,Part1,27,0.04,0.04,0.06,0.06,A,A,100,5,0.82,0.87,0.90,0.93,96
1,CPD1,Part2,26,0.03,0.02,0.04,0.04,A,A,101,5,0.87,0.88,0.84,0.89,83
2,CPD1,Part3,18,0.10,0.04,0.07,0.07,A,A,102,5,0.86,0.99,0.89,0.96,95
3,CPD1,Part4,30,0.10,0.09,0.09,0.09,A,B,103,5,0.88,0.80,0.89,0.85,96
4,CPD1,Part5,19,0.02,0.06,0.02,0.02,A,B,104,5,0.81,0.89,0.96,0.83,88
5,CPD1,Part6,14,0.04,0.04,0.10,0.10,A,B,105,5,0.93,0.82,0.97,0.99,83
6,CPD2,Part7,10,0.05,0.06,0.04,0.04,A,A,106,5,0.85,0.92,0.80,0.95,94
7,CPD2,Part8,26,0.01,0.03,0.08,0.08,A,A,107,5,0.84,0.92,0.99,0.92,84
8,CPD2,Part9,30,0.08,0.09,0.04,0.04,A,A,108,5,0.85,0.98,0.93,0.81,96
9,CPD2,Part10,12,0.05,0.04,0.03,0.03,A,B,109,5,0.84,0.85,0.91,0.95,45
